In [ ]:
import os
import cv2
import numpy as np

LOAD_FISH_EYE_CONFIG = True

VIDEO_INPUT = {
    'width': 1920,
    'height': 1080
}

# configuratie voor het ontbollen van de fisheye hoek
DIM = (1920, 1080)
K = np.array(
    [[1339.2898532184101, 0.0, 912.605852159453], [0.0, 1334.6227716562116, 540.699525078661], [0.0, 0.0, 1.0]])
D = np.array([[-0.019105044374337198], [0.26447332249046934], [-0.7164055812838268], [0.6830070634869604]])

# environment variable
FRAME_COUNT = 0
BASE_VIDEO_INPUT_PATH = './cameradata/'
BASE_OUTPUT_PATH = './undistorted_cameradata/'

VIDEO_INPUTS = {
    '3': 'Camera3_2018-08-28_14h25min25s150ms.mp4',
    '4': 'Camera4_2018-08-28_14h25min40s587ms.mp4',
    '5': 'Camera5_2018-08-28_14h25min41s707ms.mp4',
    '6': 'Camera6_2018-08-28_14h26min06s927ms.mp4',
    '7': 'Camera7_2018-08-28_14h25min42s480ms.mp4',
    '8': 'Camera8_2018-08-28_14h25min43s537ms.mp4'
}

info = []
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)

try:
    os.mkdir(BASE_OUTPUT_PATH)
except FileExistsError:
    pass

# main loop om allen camera beelden uit te pakken.
for camera, camera_name in VIDEO_INPUTS.items():

    # ophalen van het video bestand
    cap = cv2.VideoCapture(BASE_VIDEO_INPUT_PATH + camera_name)
    print(BASE_VIDEO_INPUT_PATH + camera_name)

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    output_path = '{}camera_{}.mp4'.format(BASE_OUTPUT_PATH, camera)
    print('outputing to {}'.format(output_path))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    count = 0

    while True:
        success, image = cap.read()

        if not success:
            break

        # als het beeld ontbold moet worden
        if LOAD_FISH_EYE_CONFIG:
            image = cv2.remap(image, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
            
        out.write(image)
            
        # sla elke 1000'ste frame op als png 
        if count % 1000:
            cv2.imwrite(output_path + 'img/' 'camera_{}_frame_{}.png'.format(camera, count), image)

        count += 1

    out.release()
    cap.release()